# Region and ensemble member mean table #24

Make a table by averaging values nationally and in each NCRA region for each RCM and second by computing the 10, 50, and 90 percentiles from the ensemble of RCM averaged values.
The table would have national and each NCRA region the 10,50 and 90 percentile change in the hazard metric from GWL1.2 for GWL 1.5, 2, 3

|   |rcm1	|...	|rcm13|	ens p10th	|ens p50th	|ens p90th|
|---|-------|-------|-----|-------------|-----------|---------|
|NCRA region 0|						
|...|						
|NCRA region 9|
|AUS |

Start with mean change in FFDI from GWL1.2.
Data is at /g/data/ia39/ncra/fire/ffdi for each RCM (20 years of data in each zarr collection).
Compute the average change in FFDI for each RCM and provide that in figures and tables.

In [1]:
cd /g/data/mn51/users/gt3409/plotting_maps/

/g/data/mn51/users/gt3409/plotting_maps


In [2]:
# import needed packages
from acs_area_statistics import acs_regional_stats, get_regions
import xarray as xr
import geopandas as gpd
from glob import glob
import pandas as pd
import regionmask
PATH = "/g/data/ia39/aus-ref-clim-data-nci/shapefiles/data"
import matplotlib.pyplot as plt


import dask

In [3]:
# NCRA regions from acs_area_statistics code
# these are the names of your regions
regions = get_regions([
                          "ncra_regions", "australia"
                      ])
regions

<regionmask.Regions 'ncra_regions-australia'>
overlap:  True

Regions:
0  WANorth                          Western Australia North
1  WASouth                          Western Australia South
2      NSW New South Wales and Australian Capital Territory
3      VIC                                         Victoria
4       SA                                  South Australia
5      TAS                                         Tasmania
6       NT                               Northern Territory
7 QLDNorth                                 Queensland North
8 QLDSouth                                 Queensland South
9      AUS                                        Australia

[10 regions]

In [4]:
# get the individual models for GWL30-GWL12 anomalies only
var = "FFDI"
gwl = "GWL1.5"
filelist = glob(f"/g/data/ia39/ncra/fire/change_ffdi/*{gwl}*")

In [5]:
filelist

['/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_ACCESS-CM2_ssp370_r4i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_CMCC-ESM2_ssp370_r1i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_NorESM2-MM_ssp370_r1i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_CESM2_ssp370_r11i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_CSIRO_CMCC-ESM2_ssp370_r1i1p1f1_CCAM-v2203-SN_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_MPI-ESM1-2-HR_ssp370_r1i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_CSIRO_CESM2_ssp370_r11i1p1f1_CCAM-v2203-SN_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire/change_ffdi/AGCD-05i_BOM_EC-Earth3_ssp370_r1i1p1f1_BARPA-R_v1-r1_day_GWL1.5_change_ffdi.zarr',
 '/g/data/ia39/ncra/fire

In [6]:
%%time
ds = xr.open_zarr(filelist[1], use_cftime=True)
ds

CPU times: user 5.31 s, sys: 3.26 s, total: 8.56 s
Wall time: 29.8 s


<xarray.Dataset> Size: 36GB
Dimensions:  (lat: 691, lon: 886, time: 7300)
Coordinates:
  * lat      (lat) float64 6kB -44.5 -44.45 -44.4 -44.35 ... -10.1 -10.05 -10.0
  * lon      (lon) float64 7kB 112.0 112.0 112.1 112.2 ... 156.2 156.2 156.2
  * time     (time) object 58kB 2022-01-01 12:00:00 ... 2041-12-31 12:00:00
Data variables:
    FFDI     (lat, lon, time) float64 36GB dask.array<chunksize=(33, 43, 7300), meta=np.ndarray>

In [7]:
%%time
# calculate weighted mask this is a very slow part of the code and
# can be reused for any datasets using the same regions and the same lat lon
mask_frac = regions.mask_3D_frac_approx(ds[var])

CPU times: user 17.8 s, sys: 1.83 s, total: 19.6 s
Wall time: 19.6 s


In [8]:
%%time
# calculate the area averages for each region
dict_means = {}
for file in filelist:
    ensemble_member = "-".join([file.split("/")[-1].split("_")[i] for i in (1,5,2,4)])
    print(ensemble_member)
    ds = xr.open_zarr(file, use_cftime = True,)
    ds = ds.expand_dims(dim= {"ensemble_member":[ensemble_member]}, axis=0)
    df = acs_regional_stats(ds=ds,
                                var=var, 
                                mask=mask_frac, 
                                dims = ("time", "lat", "lon",), 
                                how = ["mean"])
    # Round to two decimal places (eg for temperature anomalies)
    df[f"{var}_mean"] = df[f"{var}_mean"].round(2)

    dict_means.update({ensemble_member:df})

# dict_means = dask.compute(dict_means, traverse=False)
# merge each ensemble member into one xarray dataset
print("merging...")
ds_means = xr.merge([dict_means[key] for key in dict_means.keys()])

# calculate the ensemble median per region
print("calculating percentiles...")
ens_p10 = ds_means.quantile(0.1, dim="ensemble_member").expand_dims(dim= {"ensemble_member":["ensemble_p10"]}, axis=0).drop_vars("quantile")
ens_p50 = ds_means.quantile(0.5, dim="ensemble_member").expand_dims(dim= {"ensemble_member":["ensemble_p50"]}, axis=0).drop_vars("quantile")
ens_p90 = ds_means.quantile(0.9, dim="ensemble_member").expand_dims(dim= {"ensemble_member":["ensemble_p90"]}, axis=0).drop_vars("quantile")

# then add it to the xr dataset
print("compute...")
ds_merged_stats = xr.merge([ds_means, ens_p10, ens_p50, ens_p90]).compute()
print("convert to dataframe...")
df_means = ds_merged_stats.to_dataframe()[f"{var}_mean"].unstack().T

df_means["abbrevs"] = regions.abbrevs
df_means["names"] = regions.names
df_means = df_means[["abbrevs", "names"] + list(df_means.keys()[0:-2])]


BOM-BARPA-R-ACCESS-CM2-r4i1p1f1
BOM-BARPA-R-CMCC-ESM2-r1i1p1f1
BOM-BARPA-R-NorESM2-MM-r1i1p1f1
BOM-BARPA-R-CESM2-r11i1p1f1
CSIRO-CCAM-v2203-SN-CMCC-ESM2-r1i1p1f1
BOM-BARPA-R-MPI-ESM1-2-HR-r1i1p1f1
CSIRO-CCAM-v2203-SN-CESM2-r11i1p1f1
BOM-BARPA-R-EC-Earth3-r1i1p1f1
CSIRO-CCAM-v2203-SN-ACCESS-ESM1-5-r6i1p1f1
CSIRO-CCAM-v2203-SN-EC-Earth3-r1i1p1f1
CSIRO-CCAM-v2203-SN-ACCESS-CM2-r4i1p1f1
CSIRO-CCAM-v2203-SN-CNRM-ESM2-1-r1i1p1f2
merging...
calculating percentiles...
compute...
convert to dataframe...
CPU times: user 3h 57min 49s, sys: 1h 11min 6s, total: 5h 8min 55s
Wall time: 18min 1s


In [11]:
df_means

ensemble_member,abbrevs,names,BOM-BARPA-R-ACCESS-CM2-r4i1p1f1,BOM-BARPA-R-CESM2-r11i1p1f1,BOM-BARPA-R-CMCC-ESM2-r1i1p1f1,BOM-BARPA-R-EC-Earth3-r1i1p1f1,BOM-BARPA-R-MPI-ESM1-2-HR-r1i1p1f1,BOM-BARPA-R-NorESM2-MM-r1i1p1f1,CSIRO-CCAM-v2203-SN-ACCESS-CM2-r4i1p1f1,CSIRO-CCAM-v2203-SN-ACCESS-ESM1-5-r6i1p1f1,CSIRO-CCAM-v2203-SN-CESM2-r11i1p1f1,CSIRO-CCAM-v2203-SN-CMCC-ESM2-r1i1p1f1,CSIRO-CCAM-v2203-SN-CNRM-ESM2-1-r1i1p1f2,CSIRO-CCAM-v2203-SN-EC-Earth3-r1i1p1f1,ensemble_p10,ensemble_p50,ensemble_p90
region,,,,,,,,,,,,,,,,,
0,WANorth,Western Australia North,0.81,-0.53,2.07,-0.55,1.33,2.55,0.61,2.11,-0.51,3.32,-1.35,-0.54,-0.549,0.710,2.506
1,WASouth,Western Australia South,0.70,-0.25,1.83,-0.66,0.55,1.84,0.83,1.33,-0.17,2.80,-1.04,0.16,-0.619,0.625,1.839
2,NSW,New South Wales and Australian Capital Territory,-0.17,-0.21,1.09,-0.33,3.05,1.42,-0.10,1.89,-0.08,1.71,0.20,0.27,-0.206,0.235,1.872
3,VIC,Victoria,0.17,0.28,0.53,-0.34,1.55,0.97,-0.15,0.86,0.08,0.66,0.49,-0.06,-0.141,0.385,0.959
4,SA,South Australia,0.20,0.19,1.91,-0.54,2.25,1.65,0.45,2.09,0.02,3.39,-0.09,0.07,-0.079,0.325,2.234
5,TAS,Tasmania,0.10,0.10,0.05,-0.02,0.13,0.19,0.04,0.15,0.03,-0.01,0.13,0.04,-0.006,0.075,0.148
6,NT,Northern Territory,0.28,-0.50,1.85,-0.65,2.24,2.00,0.47,2.44,-0.52,2.80,-0.68,-0.18,-0.637,0.375,2.420
7,QLDNorth,Queensland North,0.10,0.27,1.14,-0.49,1.98,1.42,-0.04,1.72,-0.03,1.58,0.24,0.05,-0.039,0.255,1.706
8,QLDSouth,Queensland South,-0.03,0.13,1.62,-0.32,2.70,1.74,-0.00,3.02,0.06,2.43,0.05,-0.14,-0.129,0.095,2.673


In [10]:
df_means.to_csv(f"/g/data/mn51/users/gt3409/plotting_maps/ffdi_ncra-aus-regions_MME_ssp370_v1-r1_{gwl}-GWL1.2.csv")